In [14]:
from PIL import Image
import math
import matplotlib.pyplot as plt 
import os


BLACK, DARKGRAY, GRAY = ((0,0,0), (63,63,63), (127,127,127))
LIGHTGRAY, WHITE = ((191,191,191), (255,255,255))
BLUE, GREEN, RED = ((0, 0, 255), (0, 255, 0), (255, 0, 0))     
C1,C2,C3,C4 = ((20,25,31),(19,24,29),(29,36,44),(42,56,67))    #Colores para el gradiente del fondo

class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

    def rot_x(self, degrees):
        radians = math.radians(degrees)
        return self.x * math.cos(radians) + self.y * math.sin(radians)


class Rect(object):
    def __init__(self, x1, y1, x2, y2):
        minx, maxx = (x1,x2) if x1 < x2 else (x2,x1)
        miny, maxy = (y1,y2) if y1 < y2 else (y2,y1)
        self.min = Point(minx, miny)
        self.max = Point(maxx, maxy)

    def min_max_rot_x(self, degrees):
        first = True
        for x in [self.min.x, self.max.x]:
            for y in [self.min.y, self.max.y]:
                p = Point(x, y)
                rot_d = p.rot_x(degrees)
                if first:
                    min_d = rot_d
                    max_d = rot_d
                else:
                    min_d = min(min_d, rot_d)
                    max_d = max(max_d, rot_d)
                first = False
        return min_d, max_d

    width  = property(lambda self: self.max.x - self.min.x)
    height = property(lambda self: self.max.y - self.min.y)


def gradient_color(minval, maxval, val, color_palette):
    """ Calcula los colores intermedios a los definidos
    en la paleta de colores, incluyendo los extremos
    """
    max_index = len(color_palette)-1
    delta = maxval - minval
    if delta == 0:
        delta = 1
    v = float(val-minval) / delta * max_index
    i1, i2 = int(v), min(int(v)+1, max_index)
    (r1, g1, b1), (r2, g2, b2) = color_palette[i1], color_palette[i2]
    f = v - i1
    return int(r1 + f*(r2-r1)), int(g1 + f*(g2-g1)), int(b1 + f*(b2-b1))


def degrees_gradient(im, rect, color_func, color_palette, degrees):
    minval, maxval = 1, len(color_palette)
    delta = maxval - minval
    min_d, max_d = rect.min_max_rot_x(degrees)
    range_d = max_d - min_d
    for x in range(rect.min.x, rect.max.x + 1):
        for y in range(rect.min.y, rect.max.y+1):
            p = Point(x, y)
            f = (p.rot_x(degrees) - min_d) / range_d
            val = minval + f * delta
            color = color_func(minval, maxval, val, color_palette)
            im.putpixel((x, y), color)


def gradient_image(color_palette, degrees):
    region = Rect(0, 0, 1600, 900)
    width, height = region.max.x+1, region.max.y+1
    image = Image.new("RGB", (width, height), WHITE)
    degrees_gradient(image, region, gradient_color, color_palette, -degrees)
    return image






In [17]:
a = [2, 3, 4, 3, 4, 5, 3]
b = [4, 5, 5, 7, 9, 8, 6]
c = [1, 3, 9, 5, 2, 1, 1]


plt.rc('font', family='serif')

colors = [
    '#08F7FE',  # teal/cyan
    '#FE53BB',  # pink
    '#F5D300',  # yellow
    '#00ff41', # matrix green
    '#06cebc',  #Verde cyberpunk
    '#06cebc',  #naranja
    '#06cebc'   #violeta
]

cyan = (11,170,215)

color_palette = [C1,C2,C3,C4]
degrees = 305

for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
    plt.rcParams[param] = '1'


#for param in ['figure.facecolor', 'axes.facecolor', 'savefig.facecolor']:
#   plt.rcParams[param] = '#212946'  # bluish dark grey

fig, ax = plt.subplots(figsize=(16, 9), facecolor= None, frameon = False)
##ax.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
fig.suptitle('Titulo de la gráfica', fontsize=20, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#Línea Horizontal BOLD eje X
#ax.axhline(y = 0, color = 'black', linewidth = 1.3, alpha = .7)
ax.spines['bottom'].set_color('white')
ax.spines['left'].set_color('white')
ax.grid(color='grey', linestyle='-', linewidth=0.5,alpha=0.3)
#FIRMA
# ax.text(x = 0, y = -0.5,
#     s = ' ©KirilMorozov Data Scientist at Teralco',fontsize = 14, color = '#f0f0f0', backgroundcolor = 'grey')

ax.text(x = -0.7, y =-0.25,
    s = '_______________________________________________________________________________________________________________________________________________________________________________________________',
    color = 'grey', alpha = .7)

ax.text(x = -0.5, y = -0.7,
    s = '   ©KirilMorozov                                                                                                                      Source: Fuente de los Datos  ',
    fontsize = 14, color = 'grey', alpha = .6)
ax.set_facecolor('None')
ax.plot(a, marker='o',label='First Line', c = colors[1])
ax.plot(b, marker='v',label='Second Line',c = colors[0])
ax.plot(c, marker='s',label='Third Line',c = colors[3])
ax.axvspan(xmin=2, xmax=3, ymin=0, ymax=9, color='Grey', fill=True, alpha=0.3)
ax.set_title('Distance vs Workout Duration  y más explicaciones sobre la gráfica',fontsize =12)
ax.set_xlabel('Distance (Km)',fontsize =12)
ax.set_ylabel('Workout Duration (min)',fontsize =12)
leg = ax.legend(loc="upper right", edgecolor="black",frameon=False)
leg.get_frame().set_facecolor(None)
leg.set_alpha(0)
plt.close(fig)
fig.savefig('1.png', dpi= 100, )





imagen_delantera= Image.open("1.png")

background= gradient_image(color_palette, degrees)
background.paste(imagen_delantera, (0, 0), imagen_delantera)
os.remove('1.png')
background.save('1.png')
background.show()

In [ ]:
def set_plt_style():
    #Fuente para el texto
    plt.rc('font', family='serif')

    #Color del texto, de los nombres de ejers, de los ticks
    for param in ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color']:
        plt.rcParams[param] = '1'

